In [121]:
import pdb, math
import matplotlib.pyplot as plt
import numpy as np
import torch
import pdb
import cv2
plt.rcParams["figure.figsize"] = (12,12)

In [122]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy



In [142]:
resnet18 = models.resnet18(pretrained=True)
modules=list(resnet18.children())[:-1]
resnet18=nn.Sequential(*modules)

In [143]:
class SDF(nn.Module):
    def __init__(self):
        super(SDF, self).__init__()
        self.fc1 = nn.Linear(1003, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 512)
        self.fc5 = nn.Linear(512, 512)     
        self.fc6 = nn.Linear(512, 1)
        
    def forward(self, x):
#         pdb.set_trace()
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [144]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
model_ft = models.resnet18(pretrained=True)
model_ft = model_ft.to(device)

sdf_model = SDF()
# sdf_model.load_state_dict(torch.load('sdf.torch'))
sdf_model = sdf_model.to(device)


criterion = nn.MSELoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(sdf_model.parameters(), lr=0.0001)

In [145]:
from torch.utils.data import Dataset, DataLoader

class SDFDataset(Dataset):

    def __init__(self,folder):
        self.folder = folder
        self.files = os.listdir(folder)
#         pdb.set_trace()
        self.unique_ids = [x[0].split('_')[0] for x in self.files]

    def __len__(self):
        return len(self.unique_ids)

    def __getitem__(self, idx):
#         print()
        pr1 = cv2.imread(self.folder+self.unique_ids[int(idx)]+'_-1.png', cv2.IMREAD_GRAYSCALE)
        pr2 = cv2.imread(self.folder+self.unique_ids[int(idx)]+'_0.png', cv2.IMREAD_GRAYSCALE)
        pr3 = cv2.imread(self.folder+self.unique_ids[int(idx)]+'_1.png', cv2.IMREAD_GRAYSCALE)
        
        result = torch.from_numpy(np.stack([pr1,pr2,pr3])).float()/255
        return result

In [146]:
from torch.utils.data.dataloader import default_collate

In [147]:
def collate_fn(x):
    return torch.stack(x)

In [148]:
import random

from torch.utils import data
dataset = SDFDataset('/home/alex/data/3d/')

idx = dataset.unique_ids
random.shuffle(idx)  # in-place shuffle the indices to facilitate random splitting
train_idx = idx[:int(len(idx)*0.8)]
test_idx = idx[int(len(idx)*0.8):]

train_set = data.Subset(dataset, train_idx)
test_set = data.Subset(dataset, test_idx)

train_dataloader = DataLoader(train_set, batch_size=16,
                        shuffle=True, num_workers=1)
test_dataloader = DataLoader(test_set, batch_size=16,
                        shuffle=True, num_workers=1)

In [149]:
class DetectorSquare():
    def __init__(self, origin, h=100, w=120, z=0):
        self.h, self.w = h, w
        self.r = float(self.w) / self.h
        self.x1, self.x2, self.y1, self.y2, self.z1, self.z2 = -1, 1, -1. / self.r + .25, 1. / self.r + .25, z, -z
        self.xes = np.linspace(self.x1, self.x2, self.w)
        self.yes = np.linspace(self.y1, self.y2, self.h)

# origin is at -5, getector is at 5 placing objects in the middle        
detector = DetectorSquare(ORIGIN, h=H, w=W, z=5)

def get_image_with_SDF(vector, angle):
    current_points = ORIGIN.repeat(detector.w,1).view(detector.w,3).repeat(detector.h,1).view(detector.h,detector.w,3).permute(2,1,0).float().to('cuda')

    energy_map = torch.from_numpy(np.zeros((detector.w,detector.h))).float().to('cuda')
    range_map = torch.from_numpy(np.zeros((detector.w,detector.h))).float().to('cuda')

    INNER_STEP_TORCH = torch.tensor(INNER_STEP).to('cuda')
    vector_normals = (vectors.float() / norm(vectors.float())).to('cuda')
    ORIGIN_TORCH = ORIGIN.float()[:,None,None] .to('cuda')
    TORCH_ZERO = torch.tensor(0.).to('cuda')
    TORCH_ONE = torch.tensor(1.).to('cuda')

    #unit vectors from origin to detector plane
    vectors = rays - torch.from_numpy(ORIGIN.numpy()[:,None,None]).float().to('cuda')
    #starting positions over every vector
    current_points = ORIGIN.repeat(detector.w,1).view(detector.w,3).repeat(detector.h,1).view(detector.h,detector.w,3).permute(2,1,0).float()
    #placeholders
    energy_map = torch.from_numpy(np.zeros((detector.w,detector.h))).float().to('cuda')
    range_map = torch.from_numpy(np.zeros((detector.w,detector.h))).float().to('cuda')
    INNER_STEP_TORCH = torch.tensor(INNER_STEP).to('cuda')
    vector_normals = (vectors.float() / norm(vectors.float())).to('cuda')
    ORIGIN_TORCH = ORIGIN.float()[:,None,None] .to('cuda')
    for i in range(10):
        min_distance = scene.trace(current_points, angle) 
        min_distance = torch.where(min_distance>EPSILON, min_distance.float(), INNER_STEP_TORCH)
        current_points = torch_displance_point_along_vector(ORIGIN_TORCH, 
                                                            vector_normals, 
                                                            range_map.float().view(256,1)+min_distance.float()) 

        range_map  += min_distance
        energy_map += torch.where(min_distance>EPSILON, TORCH_ZERO, TORCH_ONE)
        if min_distance.min()>10: break
    pass

NameError: name 'ORIGIN' is not defined

In [150]:
def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

In [157]:
class Scene():
    def __init__(self,model):
        self.model = model
        
    def trace(self, points, angle, latent_vector):
        angle = rotation_matrix([0,0,1],angle)
        
        points = points.view(3,256)
        #FIX EINSUM
        
        #POINT OF ERROR - CHECK ME
        points = torch.einsum('iq,ij->qj',torch.from_numpy(angle).float().to('cuda'),points.to('cuda')).to('cuda')
#         pdb.set_trace()
        # latent_vector repeat 16
        # torch.cat() 
        
        return self.model(torch.cat([points.permute(1,0),latent_vector.repeat(16,1)], dim=1)).view(-1,256)

In [158]:
scene = Scene(sdf_model)

In [159]:
def norm(vectors): return torch.sqrt((vectors).pow(2).sum(0))

def torch_displace_point_along_vector(rays0, vectors, distances):
#     pdb.set_trace()
    return rays0 + (vectors.permute(1,0).repeat(16,1) * distances.float()).permute(1,0).view(3,16,16)

def trace_me(latent_vector):
    with torch.autograd.set_grad_enabled(True):
        ORIGIN = torch.tensor(np.array([0.,0.,-5.]), requires_grad=True)
        EPSILON = 0.01
        INNER_STEP = .01

        detector_points = torch.randint(0, 280, (16,2))
        current_points = ORIGIN.repeat(16,1).view(16,3).repeat(16,1).view(16,16,3).permute(2,1,0).float()

        energy_map = torch.zeros((16,16), requires_grad=True).float().to('cuda')
#         pdb.set_trace()
        range_map = torch.zeros((16,16), requires_grad=True).float().to('cuda')

        ORIGIN_TORCH = ORIGIN.float()[:,None,None].to('cuda')
        TORCH_ZERO = torch.tensor(0., requires_grad=True).to('cuda')
        TORCH_ONE = torch.tensor(1., requires_grad=True).to('cuda')
        INNER_STEP_TORCH = torch.tensor(INNER_STEP, requires_grad=True).to('cuda')

        rays = torch.cat([((detector_points+10).float()/150-1),torch.zeros(16,1)], dim=1).permute(1,0).to('cuda')
        vectors = rays - ORIGIN[:,None].float().to('cuda')

        INNER_STEP_TORCH = torch.tensor(INNER_STEP).to('cuda')
        vector_normals = (vectors.float() / norm(vectors.float())).to('cuda')
        ORIGIN_TORCH = ORIGIN.float()[:,None,None] .to('cuda')

        for i in range(10):

            min_distance = scene.trace(current_points.to('cuda'), torch.tensor(0).to('cuda'), latent_vector) *10
#             print(min_distance.mean())
            min_distance = torch.where(min_distance>EPSILON, min_distance.float(), INNER_STEP_TORCH)
        #     min_distance = torch.where(min_distance>100, min_distance.float(), torch.tensor(100))
    #         pdb.set_trace()
    #         print(current_points[:,5,5])
            current_points = torch_displace_point_along_vector(ORIGIN_TORCH, vector_normals, range_map.float().view(256,1)+min_distance.float()) 
    #         pdb.set_trace()
            range_map  += min_distance.view(16,16)
    #         pdb.set_trace()
#             pdb.set_trace()    
            energy_map += torch.where(min_distance>EPSILON, TORCH_ZERO, TORCH_ONE).view(16,16)
            if min_distance.min()>10: break
        return energy_map, detector_points
            
# ORIGIN_TORCH torch.Size([3, 1, 1])
# vector_normals torch.Size([3, 16])
# range_map torch.Size([16, 16])
# min_distance.float() torch.Size([256, 1])

In [160]:

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    sdf_model.train()
    for batch_idx, sample in enumerate(train_loader):
        
        optimizer.zero_grad()
            
        data = sample.to(device)

        output = model(data)    
        energy_map, detection_points = trace_me(output.to('cuda'))
        print(energy_map.mean())
        loss = []
        for i in range(16):
            for j in range(16):
#                 pdb.set_trace()
#                 print(data[i,1,detection_points[j,0],detection_points[j,1]]*255)
                loss.append(F.l1_loss(energy_map[i,j],data[i,1,detection_points[j,0],detection_points[j,1]]*255))
#         pdb.set_trace()
#         print(energy_map)
        loss = torch.stack(loss).mean()
        loss.backward()
#         pdb.set_trace()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, device, test_loader, optimizer, epoch, best_loss=1):
    model.eval()
    losses = []
    for batch_idx, sample in enumerate(test_loader):
        data, target = sample['coords'].to(device), sample['sdf'].to(device)
        output = model(data)
        #SDF here
        for i in output.shape[0]:
            #comapre to random angle out of 3
            image = get_image_with_SDF(output[i], angle = 0)
            
        loss = F.l1_loss(output, target)
        losses.append(loss)
#     pdb.set_trace()
    loss = sum(losses)/len(losses)
    print(f'test loss:{loss}')
    if loss<best_loss:
        torch.save(model.state_dict(), 'sdf.torch')
        best_loss = loss
    return best_loss

In [161]:
use_cuda = True

In [162]:
# optimizer = optim.Adam(model.parameters(), lr=0.00001)
best_loss = 1

for epoch in range(1, 50 + 1):
    train(model_ft, device, train_dataloader, optimizer_ft, epoch)
#     best_loss = test(model, device, test_dataloader, optimizer, epoch, best_loss)

RuntimeError: The size of tensor a (3) must match the size of tensor b (256) at non-singleton dimension 1

In [ ]:
    model.load_state_dict(torch.load('SDF.torch'))

In [25]:
import numpy as np
model = model.eval()
model(torch.from_numpy(np.array([2.3454492 , -0.96645486,  8.859472])).unsqueeze(0).float())

tensor([[4.0132]], grad_fn=<AddmmBackward>)

In [22]:
for i, x in enumerate(SDFs):
    if x<-0.2: break

In [23]:
SDFs[i]

4.0081353

In [24]:
coords[i]

array([ 2.3454492 , -0.96645486,  8.859472  ], dtype=float32)